In [286]:
from flask import Flask, request, render_template, jsonify
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pymongo
import pandas as pd

In [285]:

searchstring = 'iphone5s'
dbconnection = pymongo.MongoClient("mongodb://localhost:27017/")
database = dbconnection['Amazon_reviews']
reviews = database[searchstring].find({})
amazon_url = "https://www.amazon.in/s?k=" + searchstring
open_url = urlopen(amazon_url)
amazonpage = open_url.read()
open_url.close()
amazon_html = BeautifulSoup(amazonpage, 'html.parser')
bigbox = amazon_html.find_all('div', {'class': 'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28'})
firstbox = bigbox[0]

product_link = 'https://www.amazon.in/'+ firstbox.div.a['href']
product_open = urlopen(product_link)
product_html = BeautifulSoup(product_open, 'html.parser')
commment_box = product_html.find_all('div', {'class': 'a-section review aok-relative'})
table = database[searchstring]

reviews = []
for i in commment_box:
    try:
        name = i.find_all('span', {'class': 'a-profile-name'})[0].text
    except:
        name = 'No name'

    try:
        comment_head = i.find_all('a', {'data-hook' : 'review-title'})[0].find('span').text
    except:
        comment_head = 'No comment heading'

    try:
        ratings = i.find_all('a', {'class' : "a-link-normal"})[0].find('span').text[0:3]
    except:
        ratings = 'No ratings'

    try:
        comments = i.find_all('div', {'data-hook' : "review-collapsed"})[0].find('span').text
    except:
        comments = 'No comments'

    dictionary = {"Product": searchstring, "Name": name, "Rating": ratings, "CommentHead": comment_head, "Comment": comments}
    store_data = table.insert_one(dictionary)
    reviews.append(dictionary)



[{'Product': 'iphone5s',
  'Name': 'Ajay KS',
  'Rating': '5.0',
  'CommentHead': 'Best Value for Money in this Range. Go for it without any doubt.',
  'Comment': "\n  This is best thing you can get at this price from a reputed brand with Made in India tag. First time I ordered on the first day of online launch and am happy. I'm writing after few days of usage, but this should not be different as my experience with Samsung has been most consistent as compared to other mobile brands I've used earlier.Good:- Android 10! Beautiful looks! Finally I can delete SMS from notification now!!- Display is excellent with Blue light filter.- Performance is good. No heating or lagging. Real Racing worked fine.- 6000 Mah battery is BIG- 128 GB storage is pretty good. 512 GB card supported with dedicated external card slot.- Both dedicated sim slots support LTE- Fast charging and 15W charger top with phone. The phone came with 0% charge. Charged fast.- Mobile speakers sound is good, but I prefer exter

In [287]:
pd.DataFrame(reviews)

,Product,Name,Rating,CommentHead,Comment,_id
0,iphone5s,Ajay KS,5.0,Best Value for Money in this Range. Go for it ...,\n This is best thing you can get at this pri...,5f94f7f14851df370d244b01
1,iphone5s,shailesh,5.0,Samsung m31 all rounder best mobile.,"\n Best camera, very good battery life, eat b...",5f94f7f14851df370d244b02
2,iphone5s,surojit das,1.0,Poor camera,\n 3rd class front camera. 32 MP front camera...,5f94f7f14851df370d244b03
3,iphone5s,Riddham Jain,1.0,"False ads, poor camera quality, inefficient ba...",\n Lets just begin with the obvious. This pho...,5f94f7f14851df370d244b04
4,iphone5s,Anshul,1.0,Inferior quality and built,"\n I am using it from 3 days , 6000 mah batte...",5f94f7f14851df370d244b05
5,iphone5s,Abhinav,1.0,Serious Issue,\n The phone was wrapped in dust. Although it...,5f94f7f14851df370d244b06
6,iphone5s,abin abraham,1.0,Not good at alll,\n Software is no working. No sounds in media...,5f94f7f14851df370d244b07
7,iphone5s,Akshaykumar Patil,2.0,Samsung it's not fair .,"\n Phone camera is not good , samsung says it...",5f94f7f14851df370d244b08
